In [1]:
import os, sys
import pandas as pd
import numpy as np

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight

In [2]:
df = pd.read_csv("../feature_engineering_model/featured_datasets/final_data_featured_1.csv")
df_label = pd.read_csv("../data_gpt_labeler/labeled_datasets/final_data_labeled_1.csv")

In [3]:
df_concat = pd.concat([df,df_label], axis=1)
df_final = df_concat[["A","B","D2","E","G","policy_label"]].copy()
df_final

,A,B,D2,E,G,policy_label
0,0.126823,0.581653,0.660104,0.993421,0.999407,1
1,0.066522,0.565305,0.620909,0.360918,0.997275,1
2,0.158263,0.575038,0.551840,0.987322,0.999049,1
3,0.062633,0.575837,0.630236,0.859828,0.999422,1
4,0.005904,0.572793,0.792418,0.901759,0.999557,1
...,...,...,...,...,...,...
9995,0.353107,0.584095,0.609373,0.986010,0.999477,1
9996,0.407681,0.572810,0.561827,0.803679,0.999491,1
9997,0.129818,0.583600,0.596096,0.905910,0.999499,1
9998,0.024677,0.561784,0.500000,0.767955,0.999611,1


In [4]:
X = df_final[["A","B","D2","E"]]
y = df_final.policy_label

In [5]:
# stratified train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# calculate class_weight
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))
print("Class weights:", class_weight_dict)

Class weights: {np.int64(0): np.float64(4.587155963302752), np.int64(1): np.float64(0.5611672278338945)}


In [6]:
# MLP model
model = Sequential([
    Dense(64, input_dim=5, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # binary output
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/opt/anaconda3/envs/myenv2/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# training
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    class_weight=class_weight_dict,
    verbose=2
)

Epoch 1/20


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 5, but received input with shape (64, 4)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(64, 4), dtype=float32)
  • training=True
  • mask=None
  • kwargs=<class 'inspect._empty'>

In [ ]:
# performance
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")